In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import gc, math, os, warnings
warnings.simplefilter("ignore")
from tqdm import tqdm_notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from utils import *

In [2]:
FOLDER = "../input/"
train_df = pd.read_pickle(FOLDER + "train.pkl")
test_df = pd.read_pickle(FOLDER + "test.pkl")

building_df = pd.read_pickle(FOLDER + "building_metadata.pkl")

train_weather_df = pd.read_pickle(FOLDER + "weather_train.pkl")
test_weather_df = pd.read_pickle(FOLDER + "weather_test.pkl")

print("train/test shape is:", train_df.shape, test_df.shape)
print("weather train/test shape is:", train_weather_df.shape, test_weather_df.shape)
print("building info shape is:", building_df.shape)

train/test shape is: (20216100, 4) (41697600, 4)
weather train/test shape is: (139773, 9) (277243, 9)
building info shape is: (1449, 6)


# interpolate weather table

In [3]:
# look weather table before transformation
train_weather_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [4]:
# fill NAs
train_weather_df = fill_weather_dataset(train_weather_df)
test_weather_df = fill_weather_dataset(test_weather_df)

print("weather train/test shape is:", train_weather_df.shape, test_weather_df.shape)

# look weather table after transformation
train_weather_df.head()

KeyboardInterrupt: 

In [ ]:
# # add holiday feature
# train_weather_df = add_holiday(train_weather_df)
# test_weather_df = add_holiday(test_weather_df)

# print("weather train/test shape is:", train_weather_df.shape, test_weather_df.shape)

In [ ]:
# compute apparent temperature

def apparent_temperature(weather_df):
    
    weather_df['humidity'] = 100 - (weather_df.air_temperature - weather_df.dew_temperature) * 5
    weather_df['e'] = weather_df.humidity * 0.06105 * np.exp(17.27 * weather_df.air_temperature / (237.7 + weather_df.air_temperature))
    weather_df['apparent_temperature'] = 1.07 * weather_df.air_temperature + 0.2 * weather_df.e - 0.65 * weather_df.wind_speed - 2.7
    
    weather_df.drop(["humidity", "e"], axis=1, inplace=True) 
    
    return weather_df

train_weather_df = apparent_temperature(train_weather_df)
test_weather_df = apparent_temperature(test_weather_df)

print("weather train/test shape is:", train_weather_df.shape, test_weather_df.shape)

# look weather table after transformation
train_weather_df.head()

In [7]:
for weather_df in [train_weather_df, test_weather_df]:
    weather_df.drop(["sea_level_pressure", "wind_direction", "wind_speed"], axis=1, inplace=True) 

In [8]:
# look weather table after dropping 3 columns
train_weather_df.head()

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,timestamp,apparent_temperature
0,0,25.0,6.000000,20.0,-0.173913,2016-01-01 00:00:00,28.787444
1,0,24.4,4.285714,21.1,-1.000000,2016-01-01 01:00:00,27.522014
2,0,22.8,2.000000,21.1,0.000000,2016-01-01 02:00:00,26.761123
3,0,21.1,2.000000,20.6,0.000000,2016-01-01 03:00:00,24.743514
4,0,20.0,2.000000,20.0,-1.000000,2016-01-01 04:00:00,21.674536


In [9]:
# # create lag features for weather table
# for weather_df in [train_weather_df, test_weather_df]:
#     weather_df = create_lag_features(weather_df, 24)
# #     weather_df.drop(["air_temperature", "cloud_coverage", "dew_temperature", "precip_depth_1_hr"], axis=1, inplace=True)


In [10]:
train_weather_df.head()

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,timestamp,apparent_temperature
0,0,25.0,6.000000,20.0,-0.173913,2016-01-01 00:00:00,28.787444
1,0,24.4,4.285714,21.1,-1.000000,2016-01-01 01:00:00,27.522014
2,0,22.8,2.000000,21.1,0.000000,2016-01-01 02:00:00,26.761123
3,0,21.1,2.000000,20.6,0.000000,2016-01-01 03:00:00,24.743514
4,0,20.0,2.000000,20.0,-1.000000,2016-01-01 04:00:00,21.674536


# interpolate building table

In [5]:
# look building table before transformation
building_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [29]:
np.log1p(25000) - np.log1p(10000)

0.9162307360738424

In [26]:
BUILDING_ID = 1250
building_df[building_df.building_id==BUILDING_ID]

,site_id,building_id,primary_use,square_feet,year_built,floor_count
1250,14,1250,Education,232139,NaN,NaN


In [13]:
# encode primary_use in building_csv
le = LabelEncoder()
building_df.primary_use = le.fit_transform(building_df.primary_use)
building_df.square_feet = np.log(building_df.square_feet).astype(np.float16)

In [14]:
# look building table after transformation
building_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,8.914062,2008.0,NaN
1,0,1,0,7.910156,2004.0,NaN
2,0,2,0,8.585938,1991.0,NaN
3,0,3,0,10.070312,2002.0,NaN
4,0,4,0,11.664062,1975.0,NaN


# merge [train, weather, building] tables

In [15]:
train_df = train_df.merge(building_df, on="building_id")
train_df = train_df.merge(train_weather_df, on=["site_id", "timestamp"], how="left")
train_df = train_df[~((train_df.site_id==0) & (train_df.meter==0) & (train_df.building_id <= 104) & (train_df.timestamp < "2016-05-21"))]
train_df.reset_index(drop=True, inplace=True)
train_df.timestamp = pd.to_datetime(train_df.timestamp, format='%Y-%m-%d %H:%M:%S')
train_df["log_meter_reading"] = np.log1p(train_df.meter_reading)

test_df = test_df.merge(building_df, on="building_id")
test_df = test_df.merge(test_weather_df, on=["site_id", "timestamp"], how="left")
test_df.reset_index(drop=True, inplace=True)
test_df.timestamp = pd.to_datetime(test_df.timestamp, format='%Y-%m-%d %H:%M:%S')

In [16]:
print("After merging buiding&weather: train/test shape is:", train_df.shape, test_df.shape)

After merging buiding&weather: train/test shape is: (19867540, 15) (41697600, 14)


In [17]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,apparent_temperature,log_meter_reading
0,0,0,2016-05-21 00:00:00,249.817,0,0,8.914062,2008.0,NaN,26.7,6.00,19.4,0.0,30.303977,5.524724
1,0,0,2016-05-21 01:00:00,246.404,0,0,8.914062,2008.0,NaN,25.6,4.75,20.6,0.0,29.601180,5.511023
2,0,0,2016-05-21 02:00:00,247.087,0,0,8.914062,2008.0,NaN,26.1,4.75,20.0,0.0,28.937610,5.513779
3,0,0,2016-05-21 03:00:00,252.547,0,0,8.914062,2008.0,NaN,25.0,4.75,20.6,0.0,28.976942,5.535549
4,0,0,2016-05-21 04:00:00,244.357,0,0,8.914062,2008.0,NaN,25.0,4.75,20.6,-1.0,28.001942,5.502714


In [18]:
def fe_building_meter_agg(train_df, test_df):

    print("Begin fe aggr on building&meter&month&hour...")
    
    df_site_meter_use_hour = train_df.groupby([
        "building_id", "meter", "DT_day", "DT_hour"
    ]).agg(mean_building_meter_hour=("log_meter_reading", "mean")).reset_index()

    train_df = train_df.merge(
        df_site_meter_use_hour,
        on=["building_id", "meter", "DT_day", "DT_hour"])
    test_df = test_df.merge(df_site_meter_use_hour,
                            on=["building_id", "meter", "DT_day", "DT_hour"], how="left")
    
    return train_df, test_df

In [19]:
def fe(train_df, test_df):

    print("Begin feature engineering ...")

    # timestamp (DT_M, DT_W, DT_D, DT_hour, DT_day_week, DT_day_month, DT_week_month) 7
    for df in [train_df, test_df]:
#         df["DT_month"] = df.timestamp.dt.month.astype(np.int8) # only for group
        df['DT_hour'] = df.timestamp.dt.hour.astype(np.int8)
        df['DT_day_week'] = df.timestamp.dt.dayofweek.astype(np.int8)
        df['timestamp_og'] = df.timestamp
        df['timestamp'] = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    
    # sort data
    print("Sorting the elements ...")
    train_df = train_df.sort_values('timestamp_og')
    test_df = test_df.sort_values('timestamp_og')
    
    # remove bad rows
    print("Removing bad rows ...")
    bad_rows_idx= find_bad_zeros(train_df, train_df.log_meter_reading)
    train_df.drop(bad_rows_idx, inplace = True)
    
#     # site_id, building_id, meter <GROUP> aggregation
#     print("Grouping aggregation ...")
#     train_df, test_df = fe_building_meter_agg(train_df, test_df)
    
    # primary_use, square_feet, floor_count, year_built

    # air_temperature, dew_temperature, cloud_coverage, precip_depth_1_hr
    # create lag feature in below cell

    # sea_level_pressure, wind_direction, wind_speed <THROW>

    return train_df, test_df


train_df, test_df = fe(train_df, test_df)

Begin feature engineering ...
Sorting the elements ...
Removing bad rows ...


In [20]:
print("After merging buiding&weather: train/test shape is:", train_df.shape, test_df.shape)

train_df.head()

After merging buiding&weather: train/test shape is: (19232585, 18) (41697600, 17)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,apparent_temperature,log_meter_reading,DT_hour,DT_day_week,timestamp_og
16418015,1262,0,0.0,110.51,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,4.714114,0,4,2016-01-01
16418016,1262,1,0.0,116.00,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,4.762174,0,4,2016-01-01
16418017,1262,3,0.0,855.10,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,6.752387,0,4,2016-01-01
2639831,226,0,0.0,29.45,2,6,11.210938,1930.0,NaN,15.600000,6.00,-5.600000,0.0,11.439780,3.416086,0,4,2016-01-01
3350958,262,0,0.0,16.84,2,1,9.945312,1999.0,NaN,15.600000,6.00,-5.600000,0.0,11.439780,2.881443,0,4,2016-01-01


In [21]:
# for weather_df in [train_weather_df, test_weather_df]:
#     weather_df = create_lag_features(weather_df, 24)
# #     weather_df.drop(["air_temperature", "cloud_coverage", "dew_temperature", "precip_depth_1_hr"], axis=1, inplace=True)

# train_weather_df.timestamp = pd.to_datetime(train_weather_df.timestamp, format='%Y-%m-%d %H:%M:%S')
# test_weather_df.timestamp = pd.to_datetime(test_weather_df.timestamp, format='%Y-%m-%d %H:%M:%S')

# train_weather_df.rename(columns={"timestamp": "timestamp_og"}, inplace=True)
# test_weather_df.rename(columns={"timestamp": "timestamp_og"}, inplace=True)

# train_df = train_df.merge(train_weather_df, on=["site_id", "timestamp_og"], how="left")
# test_df = test_df.merge(test_weather_df, on=["site_id", "timestamp_og"], how="left")

In [22]:
print("After feature engineering: train/test shape is:", train_df.shape, test_df.shape)

train_df.head()

After feature engineering: train/test shape is: (19232585, 18) (41697600, 17)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,apparent_temperature,log_meter_reading,DT_hour,DT_day_week,timestamp_og
16418015,1262,0,0.0,110.51,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,4.714114,0,4,2016-01-01
16418016,1262,1,0.0,116.00,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,4.762174,0,4,2016-01-01
16418017,1262,3,0.0,855.10,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,6.752387,0,4,2016-01-01
2639831,226,0,0.0,29.45,2,6,11.210938,1930.0,NaN,15.600000,6.00,-5.600000,0.0,11.439780,3.416086,0,4,2016-01-01
3350958,262,0,0.0,16.84,2,1,9.945312,1999.0,NaN,15.600000,6.00,-5.600000,0.0,11.439780,2.881443,0,4,2016-01-01


# remove outliers

In [23]:
# building 1099 meter 2 is outlier
building_1099_idx = train_df[(train_df.building_id == 1099) & (train_df.meter == 2) & (train_df.meter_reading > 3e4)].index
train_df.drop(building_1099_idx, inplace = True)

print("After removing building 1099-2: train/test shape is:", train_df.shape, test_df.shape)

After removing building 1099-2: train/test shape is: (19229234, 18) (41697600, 17)


In [24]:
# building 778 meter 1 is outlier
building_778_idx = train_df[(train_df.building_id == 778) & (train_df.meter == 1) & (train_df.meter_reading > 2000)].index
train_df.drop(building_778_idx, inplace = True)

print("After removing building 778-1: train/test shape is:", train_df.shape, test_df.shape)

After removing building 778-1: train/test shape is: (19228395, 18) (41697600, 17)


In [25]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,apparent_temperature,log_meter_reading,DT_hour,DT_day_week,timestamp_og
16418015,1262,0,0.0,110.51,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,4.714114,0,4,2016-01-01
16418016,1262,1,0.0,116.00,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,4.762174,0,4,2016-01-01
16418017,1262,3,0.0,855.10,14,6,11.085938,NaN,NaN,3.934783,1.25,-3.282609,0.0,0.350981,6.752387,0,4,2016-01-01
2639831,226,0,0.0,29.45,2,6,11.210938,1930.0,NaN,15.600000,6.00,-5.600000,0.0,11.439780,3.416086,0,4,2016-01-01
3350958,262,0,0.0,16.84,2,1,9.945312,1999.0,NaN,15.600000,6.00,-5.600000,0.0,11.439780,2.881443,0,4,2016-01-01


In [26]:
# remove outliers [0, 0.02] and [0.98, 1]
outlier_idx = []
for meter_idx in range(4):
    for building_idx in tqdm_notebook(range(1449)):
        single_building_meter = train_df[train_df.building_id == building_idx][train_df.meter == meter_idx]
        if(len(single_building_meter)==0):
            continue
        single_building_meter_0_02 = np.round(single_building_meter.meter_reading.quantile(0.02), 4)
        single_building_meter_0_98 = np.round(single_building_meter.meter_reading.quantile(0.98), 4)
        
        outlier_low_idx = single_building_meter[single_building_meter.meter_reading < single_building_meter_0_02].index
        outlier_high_idx = single_building_meter[single_building_meter.meter_reading > single_building_meter_0_98].index
        
# #         # drop bad rows
        outlier_idx = np.concatenate([outlier_idx, outlier_low_idx, outlier_high_idx],axis=0)
        
        # replace bad rows
        train_df.loc[outlier_low_idx, 'meter_reading'] = single_building_meter_0_02
        train_df.loc[outlier_high_idx, 'meter_reading'] = single_building_meter_0_98

#         print(meter_idx, building_idx, ":", np.round(single_building_meter.meter_reading.quantile(0.05)), np.round(single_building_meter.meter_reading.quantile(0.95)))

print("train/test shape is:", train_df.shape, test_df.shape)


train/test shape is: (19228395, 18) (41697600, 17)


In [27]:
# print(outlier_idx.shape, "rows are changed ...")

# train_df.drop(outlier_idx, inplace = True)

# print("train/test shape is:", train_df.shape, test_df.shape)

In [29]:
# train_df.to_pickle("../features/train_features.pkl")
# test_df.to_pickle("../features/test_features.pkl")